In [1]:
!pwd

/home/tungdao/tung/mlopsvn/code/mlops-crash-course-code/training_pipeline/nbs


In [2]:
import pandas as pd
import fastparquet
from pathlib import Path
import numpy as np

random_seed = 17
np.random.seed(random_seed)

## Load data

In [3]:
DATA_DIR = Path("./data")
DATA_PATH = DATA_DIR / "exp_driver_stats.parquet"
LABEL_PATH = DATA_DIR / "exp_driver_orders.csv"
if not DATA_PATH.is_file():
    raise Exception("DATA_PATH not found")
if not LABEL_PATH.is_file():
    raise Exception("LABEL_PATH not found")

In [4]:
df_orig = pd.read_parquet(DATA_PATH, engine='fastparquet')
df_orig

,datetime,driver_id,conv_rate,acc_rate,avg_daily_trips,created
0,2021-07-13 11:00:00+00:00,1005,0.373837,0.154890,498,2021-07-28 11:08:04.802
1,2021-07-13 12:00:00+00:00,1005,0.571627,0.643958,656,2021-07-28 11:08:04.802
2,2021-07-13 13:00:00+00:00,1005,0.399909,0.993888,722,2021-07-28 11:08:04.802
3,2021-07-13 14:00:00+00:00,1005,0.967468,0.788458,424,2021-07-28 11:08:04.802
4,2021-07-13 15:00:00+00:00,1005,0.024679,0.956064,569,2021-07-28 11:08:04.802
...,...,...,...,...,...,...
1802,2021-07-28 09:00:00+00:00,1001,0.089418,0.311234,485,2021-07-28 11:08:04.802
1803,2021-07-28 10:00:00+00:00,1001,0.222534,0.927691,114,2021-07-28 11:08:04.802
1804,2021-04-12 07:00:00+00:00,1001,0.175219,0.761434,385,2021-07-28 11:08:04.802
902,2021-07-20 23:00:00+00:00,1003,0.025968,0.109748,55,2021-07-28 11:08:04.802


In [5]:
label_orig = pd.read_csv(LABEL_PATH, sep="\t")
label_orig

,event_timestamp,driver_id,trip_completed
0,2021-04-16 20:29:28+00:00,1001,1
1,2021-04-17 04:29:28+00:00,1002,0
2,2021-04-17 12:29:28+00:00,1003,0
3,2021-04-17 20:29:28+00:00,1001,1
4,2021-04-18 04:29:28+00:00,1002,0
5,2021-04-18 12:29:28+00:00,1003,0
6,2021-04-18 20:29:28+00:00,1001,1
7,2021-04-19 04:29:28+00:00,1002,0
8,2021-04-19 12:29:28+00:00,1003,0
9,2021-04-19 20:29:28+00:00,1004,1


### Format timestamp

In [6]:
label_orig["event_timestamp"] = pd.to_datetime(label_orig["event_timestamp"])
label_orig

,event_timestamp,driver_id,trip_completed
0,2021-04-16 20:29:28+00:00,1001,1
1,2021-04-17 04:29:28+00:00,1002,0
2,2021-04-17 12:29:28+00:00,1003,0
3,2021-04-17 20:29:28+00:00,1001,1
4,2021-04-18 04:29:28+00:00,1002,0
5,2021-04-18 12:29:28+00:00,1003,0
6,2021-04-18 20:29:28+00:00,1001,1
7,2021-04-19 04:29:28+00:00,1002,0
8,2021-04-19 12:29:28+00:00,1003,0
9,2021-04-19 20:29:28+00:00,1004,1


## Explore data

In [7]:
df_orig.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1807 entries, 0 to 902
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype              
---  ------           --------------  -----              
 0   datetime         1807 non-null   datetime64[ns, UTC]
 1   driver_id        1807 non-null   int64              
 2   conv_rate        1807 non-null   float64            
 3   acc_rate         1807 non-null   float64            
 4   avg_daily_trips  1807 non-null   int64              
 5   created          1807 non-null   datetime64[ns]     
dtypes: datetime64[ns, UTC](1), datetime64[ns](1), float64(2), int64(2)
memory usage: 98.8 KB


In [8]:
label_orig.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype              
---  ------           --------------  -----              
 0   event_timestamp  10 non-null     datetime64[ns, UTC]
 1   driver_id        10 non-null     int64              
 2   trip_completed   10 non-null     int64              
dtypes: datetime64[ns, UTC](1), int64(2)
memory usage: 368.0 bytes


## Training

### Merge features with labels

In [9]:
groups = df_orig.groupby('driver_id')

def proc_row(row):
    end_time = row['event_timestamp']
    driver_id = row['driver_id']
    grp_rows = groups.get_group(driver_id)

    # get latest record based on event_timestamp
    grp_rows = grp_rows[grp_rows['datetime'] <= end_time]
    grp_rows = grp_rows.sort_values('datetime')
    grp_rows = grp_rows.iloc[-1]
    
    # add columns
    grp_rows['event_timestamp'] = end_time
    grp_rows['trip_completed'] = row['trip_completed']

    # to Series
    return grp_rows.squeeze(axis=0)

data_df = label_orig.apply(proc_row, axis=1)
data_df

,datetime,driver_id,conv_rate,acc_rate,avg_daily_trips,created,event_timestamp,trip_completed
0,2021-04-12 07:00:00+00:00,1001,0.175219,0.761434,385,2021-07-28 11:08:04.802,2021-04-16 20:29:28+00:00,1
1,2021-04-12 07:00:00+00:00,1002,0.312347,0.481786,810,2021-07-28 11:08:04.802,2021-04-17 04:29:28+00:00,0
2,2021-04-12 07:00:00+00:00,1003,0.736727,0.936667,939,2021-07-28 11:08:04.802,2021-04-17 12:29:28+00:00,0
3,2021-04-12 07:00:00+00:00,1001,0.175219,0.761434,385,2021-07-28 11:08:04.802,2021-04-17 20:29:28+00:00,1
4,2021-04-12 07:00:00+00:00,1002,0.312347,0.481786,810,2021-07-28 11:08:04.802,2021-04-18 04:29:28+00:00,0
5,2021-04-12 07:00:00+00:00,1003,0.736727,0.936667,939,2021-07-28 11:08:04.802,2021-04-18 12:29:28+00:00,0
6,2021-04-12 07:00:00+00:00,1001,0.175219,0.761434,385,2021-07-28 11:08:04.802,2021-04-18 20:29:28+00:00,1
7,2021-04-12 07:00:00+00:00,1002,0.312347,0.481786,810,2021-07-28 11:08:04.802,2021-04-19 04:29:28+00:00,0
8,2021-04-12 07:00:00+00:00,1003,0.736727,0.936667,939,2021-07-28 11:08:04.802,2021-04-19 12:29:28+00:00,0
9,2021-04-12 07:00:00+00:00,1004,0.094609,0.151163,166,2021-07-28 11:08:04.802,2021-04-19 20:29:28+00:00,1


In [10]:
data_df = data_df[data_df.columns. \
    drop("datetime"). \
    drop("driver_id"). \
    drop("created"). \
    drop("event_timestamp")]
data_df

,conv_rate,acc_rate,avg_daily_trips,trip_completed
0,0.175219,0.761434,385,1
1,0.312347,0.481786,810,0
2,0.736727,0.936667,939,0
3,0.175219,0.761434,385,1
4,0.312347,0.481786,810,0
5,0.736727,0.936667,939,0
6,0.175219,0.761434,385,1
7,0.312347,0.481786,810,0
8,0.736727,0.936667,939,0
9,0.094609,0.151163,166,1


### Split data

In [11]:
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [12]:
def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2

In [13]:
selected_ft = ["conv_rate", "acc_rate", "avg_daily_trips"]
TARGET_COL = "trip_completed"
TEST_SIZE = 0.2

train, test = train_test_split(data_df, test_size=TEST_SIZE, random_state=random_seed)
train_x = train.drop([TARGET_COL], axis=1)[selected_ft]
test_x = test.drop([TARGET_COL], axis=1)[selected_ft]
train_y = train[[TARGET_COL]]
test_y = test[[TARGET_COL]]
train_x.shape, train_y.shape, test_x.shape, test_y.shape

((8, 3), (8, 1), (2, 3), (2, 1))

### Training and evaluation

In [14]:
ALPHA = 0.5
L1_RATIO = 0.1

model = ElasticNet(alpha=ALPHA, l1_ratio=L1_RATIO, random_state=random_seed)
model.fit(train_x, train_y)

predicted_qualities = model.predict(test_x)
(rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (ALPHA, L1_RATIO))
print("  RMSE: %s" % rmse)
print("  MAE: %s" % mae)
print("  R2: %s" % r2)

Elasticnet model (alpha=0.500000, l1_ratio=0.100000):
  RMSE: 0.11625695183915705
  MAE: 0.10880469049979313
  R2: 0.0


### Export, import and inference

In [15]:
import joblib
MODEL_DIR = Path("./model")
MODEL_DIR.mkdir(parents=True, exist_ok=True)
model_path = MODEL_DIR / "driver_model.bin"
joblib.dump(model, model_path)

['model/driver_model.bin']

In [16]:
loaded_model = joblib.load(model_path)
loaded_model

ElasticNet(alpha=0.5, l1_ratio=0.1, random_state=17)

In [17]:
predictions = loaded_model.predict(test_x)
predictions

array([ 0.14975854, -0.06785084])